In [44]:
import hashlib
import random
import simple

H = hashlib.sha256

def H(*args):  
    a = ':'.join(str(a) for a in args)
    return int(hashlib.sha256(a.encode('utf-8')).hexdigest(), 16)

def cryptrand(n=1024):
    return random.SystemRandom().getrandbits(n) % N

def isPrime(n):
    if n % 2 == 0:
        return n == 2
    d = 3
    while d * d <= n and n % d != 0:
        d += 2
    return d * d > n

def generator(N):
    p = N-1
    q = 0
    j = 0
    for i in range(2, N, 2):
        if isPrime(p//i):
            q = p//i
            j = i
            break
    for h in range(2, p):
        g = pow(h, j, N)
        if g > 1:
            return g
  


In [59]:
N = 0
q = 0
while(True):
    N = simple.simpleNum(50)
    q = (N-1)//2
    if isPrime(q):
        break
print(N)
print(q)

694886463273083
347443231636541


In [60]:
g = generator(N)
print(g)

4


In [61]:
# параметр
k = H(N, g)
k

73941764427541727585766948720092387360520602990087869046095446434437179334183

In [62]:
# the server must first generate the password verifier
I = "YMandron"         # Username
p = "qwerty123456"   # Password

s = cryptrand(64)    # Salt for the user
x = H(s, I, p)       # Private key
v = pow(g, x, N)     # Password verifier

In [63]:
# 1. client sends username I and public ephemeral value A to the server
a = cryptrand()
A = pow(g, a, N)

In [64]:
# 2. server sends user's salt s and public ephemeral value B to client
b = cryptrand()
B = (k * v + pow(g, b, N)) % N

In [65]:
# 3. client and server calculate the random scrambling parameter
u = H(A, B)

In [66]:
# 4. client computes session key
x = H(s, I, p)
S_c = pow(B - k * pow(g, x, N), a + u * x, N)
K_c = H(S_c)

In [67]:
# 5. server computes session key
S_s = pow(A * pow(v, u, N), b, N)
K_s = H(S_s)

In [68]:
# 6. client sends proof of session key to server
M_c = H(H(N) ^ H(g), H(I), s, A, B, K_c)

In [69]:
# 7. server sends proof of session key to client
M_s = H(A, M_c, K_s)

In [70]:
print(K_s)
print(K_c)

61052546598754862816887244092662914807473740382133124215407061157203671146200
61052546598754862816887244092662914807473740382133124215407061157203671146200


In [71]:
print(M_s)
print(M_c)

26666070696603291072888954881374367892974139720031876446123807554019995449155
104303472649190640946466945081202738430490900814345422549886838598488183456079


In [72]:
print(H(A, M_c, K_c))
print(H(B, M_s, K_s))

26666070696603291072888954881374367892974139720031876446123807554019995449155
87790813410653586566866177426080354711919584762870212193541917242306673217920
